In [13]:
# импорт зависимосте

import pandas as pd
import numpy as np
import xarray as xr


import psycopg

In [26]:
def req(y0, y1):
    return """
                       select dt, speed, temperature, density
                       from rss.ace_at_earth_1h
                       where (
                       ace_at_earth_1h.dt >= DATE '{0}-01-01'
                       and
                       ace_at_earth_1h.dt <= DATE '{1}-01-01'
                       )
""".format(
        y0, y1
    )

In [27]:
years = list(range(2010, 2025))
years

[2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023,
 2024]

In [28]:
reqs = []

for i in range(len(years) - 1):
    reqs.append(req(years[i], years[i + 1]))
reqs

["\n                       select dt, speed, temperature, density\n                       from rss.ace_at_earth_1h\n                       where (\n                       ace_at_earth_1h.dt >= DATE '2010-01-01'\n                       and\n                       ace_at_earth_1h.dt <= DATE '2011-01-01'\n                       )\n",
 "\n                       select dt, speed, temperature, density\n                       from rss.ace_at_earth_1h\n                       where (\n                       ace_at_earth_1h.dt >= DATE '2011-01-01'\n                       and\n                       ace_at_earth_1h.dt <= DATE '2012-01-01'\n                       )\n",
 "\n                       select dt, speed, temperature, density\n                       from rss.ace_at_earth_1h\n                       where (\n                       ace_at_earth_1h.dt >= DATE '2012-01-01'\n                       and\n                       ace_at_earth_1h.dt <= DATE '2013-01-01'\n                       )\n",
 

In [29]:
from userpwd import userpwd_postgre

In [30]:
res = []

conn = psycopg.connect(
    host="213.131.1.41", user="selector", dbname="smdc", password=userpwd_postgre
)

In [31]:
with conn.cursor() as cur:
    for req in reqs:
        print(req)
        for row in cur.execute(req):
            res.append(row)



                       select dt, speed, temperature, density
                       from rss.ace_at_earth_1h
                       where (
                       ace_at_earth_1h.dt >= DATE '2010-01-01'
                       and
                       ace_at_earth_1h.dt <= DATE '2011-01-01'
                       )


                       select dt, speed, temperature, density
                       from rss.ace_at_earth_1h
                       where (
                       ace_at_earth_1h.dt >= DATE '2011-01-01'
                       and
                       ace_at_earth_1h.dt <= DATE '2012-01-01'
                       )


                       select dt, speed, temperature, density
                       from rss.ace_at_earth_1h
                       where (
                       ace_at_earth_1h.dt >= DATE '2012-01-01'
                       and
                       ace_at_earth_1h.dt <= DATE '2013-01-01'
                       )


                       select dt, sp

In [36]:
df = pd.DataFrame(
        np.array(res), columns=["date", "speed", "temperature", "density"]
    )

df["date"] = pd.to_datetime(df["date"])

df.set_index("date", inplace=True)

df

,speed,temperature,density
date,,,
2010-09-15 15:00:00,355.225833,68982.5,1.216667
2010-10-04 15:00:00,292.826,31213.717949,0.478333
2010-10-30 02:00:00,336.010976,88706.153846,0.1
2010-08-16 03:00:00,367.921667,64710.833333,1.191667
2010-08-03 10:00:00,411.98,25764.166667,3.238333
...,...,...,...
2023-10-17 19:00:00,292.905085,12259.333333,2.75113
2023-08-22 05:00:00,511.002542,99088.135593,1.447175
2023-09-27 23:00:00,441.466949,118648.305085,1.051977


In [ ]:
df.to_parquet("../Data/ace_at_earth_1h.parquet")